In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

本文前面的部分介绍了现代计算机视觉网络设计的发展历程。这些工作的共同特点在于它们在很大程度上依赖于科学家的直觉。许多网络架构都是依赖人类创造力设计的，而不是系统地探索深度网络所提供的设计空间。尽管如此，这种网络工程方法取得了巨大的成功。

自从AlexNet（第8.1节）在ImageNet上击败了传统的计算机视觉模型，通过堆叠卷积块来构建非常深的网络变得流行起来，这些卷积块都是根据相同的模式设计的。特别是，VGG网络（第8.2节）使卷积变得流行。NiN（第8.3节）表明，通过添加局部非线性，甚至卷积也可能有益。此外，NiN通过在所有位置聚合信息来解决网络头部聚合信息的问题。GoogLeNet（第8.4节）添加了不同卷积宽度的多个分支，在其Inception块中结合了VGG和NiN的优点。ResNet（第8.6节）将归纳偏差改为恒等映射（从卷积）。这允许非常深的网络。将近十年后，ResNet设计仍然受欢迎，这充分证明了其设计的优越性。最后，ResNeXt（第8.6.5节）添加了分组卷积，提供了参数和计算之间更好的权衡。Squeeze-and-Excitation Networks（SENets）作为视觉变压器的先驱，允许在位置之间进行高效的信息传输（Hu等人，2018）。这是通过计算每个通道的全局注意力函数来实现的。

到目前为止，我们还没有讨论通过神经架构搜索（NAS）（Liu等人，2018，Zoph和Le，2016）获得的网络。我们选择这样做，因为它们的成本通常非常高，依赖于暴力搜索、遗传算法、强化学习或其他形式的超参数优化。在固定的搜索空间中，NAS使用搜索策略根据返回的性能估计自动选择架构。NAS的结果是一个单独的网络实例。EfficientNets是这种搜索的一个显著结果（Tan和Le，2019）。

接下来，我们将讨论一个与寻求单个最佳网络截然不同的想法。它在计算上相对便宜，过程中可以获得科学洞见，并且在结果质量方面非常有效。让我们回顾一下Radosavovic等人（2020）设计网络设计空间的策略。该策略结合了手动设计和NAS的优势。它通过操作网络分布并以一种获得整个网络族的良好性能的方式优化分布来实现这一点。它的结果是RegNets，特别是RegNetX和RegNetY，以及一系列有关设计高性能CNN的指导原则。

下面的描述紧密遵循Radosavovic等人（2020）的推理，以适应本书的范围。首先，我们需要为要探索的网络族提供一个模板。本章设计的共性之一是网络由一个stem（起始部分）、一个body（主体部分）和一个head（顶部部分）组成。stem部分通过较大窗口大小的卷积进行初始图像处理。body部分由多个块组成，执行从原始图像到对象表示所需的大部分变换。最后，head部分将其转换为所需的输出，例如通过用于多类分类的softmax回归器。另一方面，body部分由多个阶段组成，以降低图像分辨率。实际上，stem部分和每个后续阶段都将空间分辨率减小四分之一。最后，每个阶段由一个或多个块组成。这种模式适用于所有网络，从VGG到ResNeXt。实际上，为了设计通用的AnyNet网络，Radosavovic等人（2020）使用了图8.6.5中的ResNeXt块。

总结：网络设计模板包括stem（起始部分）、body（主体部分，包含多个阶段）和head（顶部部分）。每个阶段由一个或多个块组成，这种模式适用于从VGG到ResNeXt的所有网络。对于通用的AnyNet网络设计，Radosavovic等人（2020）使用了ResNeXt块。

In [2]:
class AnyNet(d2l.Classifier):
    def stem(self, num_channels):
        return nn.Sequential(
            nn.LazyConv2d(num_channels, kernel_size=3, stride=2, padding=1),
            nn.LazyBatchNorm2d(), nn.ReLU())

In [3]:
@d2l.add_to_class(AnyNet)
def stage(self, depth, num_channels, groups, bot_mul):
    blk = []
    for i in range(depth):
        if i == 0:
            blk.append(d2l.ResNeXtBlock(num_channels, groups, bot_mul,
                                        use_1x1conv=True, strides=2))
        else:
            blk.append(d2l.ResNeXtBlock(num_channels, groups, bot_mul))
    return nn.Sequential(*blk)

In [4]:
@d2l.add_to_class(AnyNet)
def __init__(self, arch, stem_channels, lr=0.1, num_classes=10):
    super(AnyNet, self).__init__()
    self.save_hyperparameters()
    self.net = nn.Sequential(self.stem(stem_channels))
    for i, s in enumerate(arch):
        self.net.add_module(f'stage{i+1}', self.stage(*s))
    self.net.add_module('head', nn.Sequential(
        nn.AdaptiveAvgPool2d((1, 1)), nn.Flatten(),
        nn.LazyLinear(num_classes)))
    self.net.apply(d2l.init_cnn)

In [5]:
class RegNetX32(AnyNet):
    def __init__(self, lr=0.1, num_classes=10):
        stem_channels, groups, bot_mul = 32, 16, 1
        depths, channels = (4, 6), (32, 80)
        super().__init__(
            ((depths[0], channels[0], groups, bot_mul),
             (depths[1], channels[1], groups, bot_mul)),
            stem_channels, lr, num_classes)

In [6]:
RegNetX32().layer_summary((1, 1, 96, 96))

/data2/miniconda3/envs/d2l-pytorch/lib/python3.8/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Sequential output shape:	 torch.Size([1, 32, 48, 48])
Sequential output shape:	 torch.Size([1, 32, 24, 24])
Sequential output shape:	 torch.Size([1, 80, 12, 12])
Sequential output shape:	 torch.Size([1, 10])


In [7]:
model = RegNetX32(lr=0.05)
trainer = d2l.Trainer(max_epochs=10, num_gpus=1)
data = d2l.FashionMNIST(batch_size=128, resize=(96, 96))
trainer.fit(model, data)

RuntimeError: GET was unable to find an engine to execute this computation

In [ ]:
'''8_8_designing_convolution_network_architectures.ipynb
具有良好归纳偏置（假设或偏好）的卷积神经网络（CNNs），如局部性和平移不变性（第7.1节）对于视觉任务，一直是这个领域的主导架构。
从LeNet到Transformers（第11.7节）（Dosovitskiy等人，2021；Touvron等人，2021）开始在准确性方面超过CNNs，情况一直如此。
尽管最近在视觉Transformers方面取得的进展可以移植到CNNs（Liu等人，2022），但只能以更高的计算成本实现。
同样重要的是，最近的硬件优化（NVIDIA Ampere和Hopper）只是扩大了Transformers的优势。

值得注意的是，与CNNs相比，Transformers对局部性和平移不变性的归纳偏置程度要低得多。
学习到的结构优势不仅仅归功于大型图像集合的可用性，如LAION-400m和LAION-5B（Schuhmann等人，
2022），这些图像集合最多可达50亿张图像。令人惊讶的是，在这方面的一些更相关的工作甚至包括MLPs（Tolstikhin等人，2021）。

总之，视觉Transformers（第11.8节）现在在大规模图像分类方面的最先进性能中处于领先地位，
表明可伸缩性优于归纳偏置（Dosovitskiy等人，2021）。这包括使用多头自注意力（第11.5节）
预训练大规模Transformers（第11.9节）。我们邀请读者深入阅读这些章节，以进行更详细的讨论。
'''

'''
归纳偏置（Inductive Bias）是指机器学习算法在学习过程中对某些假设或先验知识的依赖。
归纳偏置可以看作是一种启发式，它有助于在训练数据不足或不完整的情况下引导模型学习更合适的解决方案。
换句话说，归纳偏置是一种对问题的偏好，它可以帮助模型在面对多种可能解决方案时，选择更可能的那个。

在深度学习中，归纳偏置通常体现在模型架构的设计中。
例如，卷积神经网络（CNNs）具有局部性和平移不变性的归纳偏置。这意味着CNNs在处理图像数据时，
更倾向于关注局部特征并在不同位置保持相同的响应。这些偏置使得CNNs在处理视觉任务时具有很好的性能。

另一方面，Transformers具有较低的归纳偏置，这意味着它们在学习过程中对先验知识的依赖较小。
这使得Transformers在大量数据可用时能够学习到更丰富的表示，但可能需要更多的计算资源和数据来达到相同的性能水平。

总之，归纳偏置是机器学习算法在学习过程中对某些假设或先验知识的依赖。
选择合适的归纳偏置对于提高模型性能和泛化能力至关重要。
在深度学习中，不同的模型架构具有不同程度的归纳偏置，这影响了它们在处理特定任务时的性能。 
'''

1. Increase the number of stages to four. Can you design a deeper RegNetX that performs better?

2. De-ResNeXt-ify RegNets by replacing the ResNeXt block with the ResNet block. How does your new model perform?

3. Implement multiple instances of a “VioNet” family by violating the design principles of RegNetX. How do they perform? Which of (d_i, c_i, g_i,  b_i) is the most important factor?

4. Your goal is to design the “perfect” MLP. Can you use the design principles introduced above to find good architectures? Is it possible to extrapolate from small to large networks?


1. 增加阶段数量到四个。您能设计一个性能更好的更深的RegNetX吗？

   要设计一个更深的RegNetX，您可以增加每个阶段的层数。例如，您可以尝试将每个阶段的层数翻倍。在增加层数的同时，您可能还需要调整其他超参数，例如宽度和深度。为了找到性能更好的RegNetX，您可以使用网格搜索或贝叶斯优化等技术来搜索最佳超参数组合。

2. 通过将ResNeXt块替换为ResNet块，对RegNets进行去ResNeXt化。您的新模型表现如何？

   将ResNeXt块替换为ResNet块后，您的新模型将失去一些分组卷积的优势，可能导致性能略有下降。然而，这种变化可能会降低模型的参数数量和计算成本。为了评估新模型的性能，您需要在相应的数据集上进行训练和测试。

3. 通过违反RegNetX的设计原则，实现多个“VioNet”家族的实例。它们的性能如何？哪个因素（d_i，c_i，g_i，b_i）最重要？

   要创建VioNet家族的实例，您可以尝试违反RegNetX的设计原则，例如使用不同的深度、宽度、分组和瓶颈比例。在实验中，您可能会发现某些因素对性能影响更大。通常，深度（d_i）和宽度（c_i）可能是最重要的因素，因为它们直接影响模型的表示能力。分组（g_i）和瓶颈比例（b_i）也可能影响性能，但它们主要影响计算成本和参数数量。

4. 您的目标是设计“完美”的MLP。您能否使用上面介绍的设计原则找到良好的架构？从小型网络到大型网络是否可以进行外推？

   要设计一个“完美”的MLP，您可以尝试应用类似于RegNetX的设计原则。例如，您可以使用不同的层数、隐藏单元数量和激活函数来找到最佳架构。但是，MLP与CNN（如RegNetX）在结构上有很大不同，因此这些设计原则可能需要相应地调整。

   从小型网络到大型网络进行外推可能具有一定的挑战性。在某些情况下，较大的网络可能会表现更好，因为它们具有更强的表示能力。然而，这并不总是成立，因为较大的网络可能更容易过拟合，并且需要更多的计算资源和训练时间。为了找到适合不同任务的最佳网络架构，可能需要进行大量的实验和超参数搜索。